In [297]:
import random
import math
import csv
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

In [298]:
collection_of_sequences = []

# No. of components
no_of_components = 16

#Directions / Orientations
orientations = []
orientations = ['-z','-z','-z','-z','-z','+z','-z','+x','-z','-y','-z','-y']
#Tool Grippers
tools_grippers = []
tools_grippers = ['A','A','A','A','A','A','B','C','D','E','F','G']

#Precedence Matrix
if no_of_components==4:
    file = 'precedence_matrix.csv'
elif no_of_components==12:
    file = 'precedence_matrix_12.csv'
elif no_of_components==16:
    file = 'precedence_matrix_16.csv'
else:
    print("Error!!")
    
#Directions / Orientations
if no_of_components==4:
    orientations = ['+z','-z','+z','-z']
elif no_of_components==12:
    orientations = ['-z','-z','-z','-z','-z','+z','-z','+x','-z','-y','-z','-y']
elif no_of_components==16:
    orientations = ['-z','+x','-z','-z','-z','-z','-z','-z','-z','-z','+y','+y','-y','-y','+x','+x']
else:
    print("Error!!")

#Tool Grippers
if no_of_components==4:
    tools_grippers = ['A','B','C','A']
elif no_of_components==12:
    tools_grippers = ['A','A','A','A','A','A','B','C','D','E','F','G']
elif no_of_components==16:
    tools_grippers = ['A','B','C','C','D','D','D','D','E','E','E','E','E','E','F','F']
else:
    print("Error!!")
    
precedence_list = []
with open(file) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        precedence_list.append(row)

print(orientations)
print(tools_grippers)
print(precedence_list)

['-z', '+x', '-z', '-z', '-z', '-z', '-z', '-z', '-z', '-z', '+y', '+y', '-y', '-y', '+x', '+x']
['A', 'B', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'E', 'E', 'F', 'F']
[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '0', '1', '1'], ['1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], [

In [299]:
wt = 0.5 #random.uniform(0, 1)
wor = 0.5 #random.uniform(0, 1)    

def check_precedence_criteria(sequence):
    for i,part in enumerate(sequence):
        to_search = precedence_list[part]
        done_sequence = sequence[0:i]
        to_search_final = [x for j, x in enumerate(to_search) if j not in done_sequence]
        for a in to_search_final:
            if (a == '1'):
                return False
    return True

def calc_orientation_changes(sequence):
    orien_changes = 0
    current_orientation = orientations[sequence[0]]
    for part in sequence[1:]:
        if orientations[part] != current_orientation:
            orien_changes += 1
            current_orientation = orientations[part]
    return orien_changes

def calc_tool_changes(sequence):
    tool_changes = 0
    current_tool = tools_grippers[sequence[0]]
    for part in sequence[1:]:
        if tools_grippers[part] != current_tool:
            tool_changes += 1
            current_tool = tools_grippers[part]
    return tool_changes

def fitness_function(sequence):
    is_feasible = check_precedence_criteria(sequence)
    feasibility_index = 0.0
    if is_feasible == True:
        feasibility_index = 2.0
    nt = calc_tool_changes(sequence)
    nor = calc_orientation_changes(sequence)
    FF = 1/(wor*nor + wt*nt - feasibility_index) 
    return FF

In [300]:
def update_velocity(particle, velocity, pbest, gbest, w_min=0.5, max=1.0, c=0.5):
    # Initialise new velocity array
    num_particle = len(particle)
    new_velocity = np.array([0.0 for i in range(num_particle)])

    # Randomly generate r1, r2 and inertia weight from normal distribution
    r1 = random.uniform(0,0.2)
    r2 = random.uniform(0,0.2)
    w = random.uniform(w_min,0.8)
    c1 = c
    c2 = c
    # Calculate new velocity
    for i in range(num_particle):
        new_velocity[i] = w*velocity[i] + c1*r1*(pbest[i]-particle[i])+c2*r2*(gbest[i]-particle[i])
        if new_velocity[i] > 1:
            new_velocity[i] = random.uniform(0, 0.8)
    return new_velocity

In [301]:
def update_position(particle, velocity):
    # Move particles by adding velocity
    new_particle = particle + velocity
    particle = [int(x) for x in new_particle] 
    visited = [False for i in range(len(particle))]
    not_visited = []
    repair = []
    for j in range(len(particle)):
        num = particle[j]
        if num < no_of_components and num >= 0 and visited[num] == False:
            visited[num] = True
        else:
            repair.append(j)
    for j in range(no_of_components):
        if visited[j] == False:
            not_visited.append(j)
    for j in range(len(repair)):
        num = random.choice(not_visited)
        index = repair[j]
        particle[index] = num
        visited[num] = True
        not_visited.remove(num)
    return particle

In [302]:
collection_of_best_sequences = []
collection = []

def pso(population, generation):
    # Initialisation
    # Population
    particles = [[random.randint(0, no_of_components-1) for j in range(no_of_components)] for i in range(population)]
    
#     # Correction
    for i in range(population):
#         print(particles[i])
        visited = [False for i in range(no_of_components)]
        not_visited = []
        repair = []
        for j in range(no_of_components):
            num = particles[i][j]
            if num < no_of_components and num >= 0 and visited[num] == False:
                visited[num] = True
            else:
                repair.append(j)
        for j in range(no_of_components):
            if visited[j] == False:
                not_visited.append(j)
#         print(not_visited)
        for j in range(len(repair)):
            num = random.choice(not_visited)
            index = repair[j]
            particles[i][index] = num
            visited[num] = True
            not_visited.remove(num)
#         print(particles[i], "\n")
    
    # Particle's best position
    pbest_position = particles
    # Fitness
    pbest_fitness = [fitness_function(p) for p in particles]
    
    # Index of the best particle
    gbest_index = np.argmax(pbest_fitness)
    # Global best particle position
    gbest_position = pbest_position[gbest_index]
    # Velocity (starting from 0 speed)
    velocity = [[random.uniform(0,0.5) for j in range(no_of_components)] for i in range(population)]

    # Loop for the number of generation
    for t in range(generation):
    # Stop if the average fitness value reached a predefined success criterion
        for n in range(population):
            # Update the velocity of each particle
            velocity[n] = update_velocity(particles[n], velocity[n], pbest_position[n], gbest_position)
            # Move the particles to new position
            particles[n] = update_position(particles[n], velocity[n])
            if(fitness_function(particles[n]) > fitness_function(pbest_position[n])):
                pbest_position[n] = particles[n]
        # Calculate the fitness value
        pbest_fitness = [fitness_function(p) for p in pbest_position]
        # Find the index of the best particle
        gbest_index = np.argmax(pbest_fitness)
        # Update the position of the best particle
        gbest_position = pbest_position[gbest_index]

    # Print the results
    print('Global Best Sequence: ', gbest_position)
    print('Precedence criteria: ', check_precedence_criteria(gbest_position))
    print('No. of tool changes: ', calc_tool_changes(gbest_position))
    print('No. of orientation changes: ', calc_orientation_changes(gbest_position))
    print('Best Fitness Value: ', min(pbest_fitness))
    print('Average Particle Best Fitness Value: ', np.average(pbest_fitness))
    return calc_tool_changes(gbest_position) + calc_orientation_changes(gbest_position)
#     return [calc_tool_changes(gbest_position), calc_orientation_changes(gbest_position)]

no_of_iterations = 2000
# total_no_of_tool_orientation_changes = 1e9

# for i in range(100):
pso(50, no_of_iterations)

Global Best Sequence:  [0, 1, 2, 3, 4, 7, 6, 5, 8, 9, 12, 13, 10, 11, 14, 15]
Precedence criteria:  False
No. of tool changes:  5
No. of orientation changes:  5
Best Fitness Value:  0.2
Average Particle Best Fitness Value:  0.19999999999999996


10

In [303]:
# tlArr, ortArr = [],[]
# for i in range(100):
#     tl, ort = pso(no_of_components, no_of_iterations)
#     tlArr.append(tl)
#     ortArr.append(ort)
# import numpy as np
# print(np.mean(tlArr))
# print(np.std(tlArr))
# print(np.mean(ortArr))
# print(np.std(ortArr))

# print("range tl: ", np.mean(tlArr) - 2.576*np.std(tlArr), np.mean(tlArr) + 2.576*np.std(tlArr))
# print("range ort: ", np.mean(ortArr) - 2.576*np.std(ortArr), np.mean(ortArr) + 2.576*np.std(ortArr))

In [309]:
s = [1, 4, 3, 6, 5, 8, 7, 10, 9, 16, 15, 2, 12, 11, 13, 14]
for i in range(len(s)):
    s[i] = s[i] - 1
print(check_precedence_criteria(s))
print(calc_tool_changes(s))
print(calc_orientation_changes(s))
print(fitness_function(s), "\n")

True
6
3
0.4 



In [305]:
s = [0, 4, 2, 1, 10, 6, 7, 3, 8, 11, 5, 9]
print(check_precedence_criteria(s))
print(calc_tool_changes(s))
print(calc_orientation_changes(s))
print(fitness_function(s), "\n")

False
9
5
0.14285714285714285 

